In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tableone import TableOne
from scipy.stats import shapiro
import math
from geopy.distance import geodesic

ModuleNotFoundError: No module named 'matplotlib'

In [11]:
!pip install tableone > null



[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
url = 'https://raw.githubusercontent.com/4GeeksAcademy/data-preprocessing-project-tutorial/main/AB_NYC_2019.csv'

In [ ]:
df = pd.read_csv(url)

In [ ]:
# df.duplicated().sum()
#df.shape
df.dtypes

In [ ]:
# comprobar si la variable binaria o es categorica

def resumen_estadistico_categoricas(df):
    resumen = []

    # Seleccionar variables categóricas
    columnas_cat = df.select_dtypes(include=['object', 'category', 'bool']).columns

    for col in columnas_cat:
        serie = df[col]
        total = len(serie)
        n_missing = serie.isna().sum()
        n_unicos = serie.nunique(dropna=True)

        # Moda y su frecuencia
        if serie.dropna().empty:
            moda = None
            frec_moda = 0
        else:
            moda = serie.mode().iloc[0]
            frec_moda = (serie == moda).sum()

        resumen.append({
            'Var': col,
            'Tipo': 'Binaria' if n_unicos == 2 else 'Cat múlt',
            'Moda': moda,
            'Frec moda': frec_moda,
            '% moda': round((frec_moda / total) * 100, 2),
            'Val únicos': n_unicos,
            'Faltantes': n_missing,
            '% faltantes': round((n_missing / total) * 100, 2)
        })

    return pd.DataFrame(resumen)

In [ ]:
resumen = resumen_estadistico_categoricas(df)
print(resumen)

In [ ]:
#Distancia entre puntos turisticos

def distancia_haversine(lat1, lon1, lat2, lon2):
    r = 6371  # Radio de la Tierra en km
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return r * c


central_lat =  40.785091
central_lon = -73.968285

df['central_park'] = df.apply(
    lambda row: distancia_haversine(
        central_lat, central_lon,
        row['latitude'], row['longitude']
    ),
    axis=1
)



time_lat =  40.7580
time_lon = -73.9855

df['time_square'] = df.apply(
    lambda row: distancia_haversine(
        time_lat, time_lon,
        row['latitude'], row['longitude']
    ),
    axis=1
)


estatua_lat =  40.6892
estatua_lon = -74.0445

df['estatua_libertad'] = df.apply(
    lambda row: distancia_haversine(
        estatua_lat, estatua_lon,
        row['latitude'], row['longitude']
    ),
    axis=1
)



broadway_lat =  40.7590
broadway_lon = -73.9845

df['broadway'] = df.apply(
    lambda row: distancia_haversine(
        broadway_lat, broadway_lon,
        row['latitude'], row['longitude']
    ),
    axis=1
)


In [ ]:
#df.info()
df['room_type'].unique()

In [ ]:
df['neighbourhood_group'].unique()

In [ ]:
df.isna().sum()

In [ ]:
df.groupby('room_type')[['price']].agg(['mean', 'std'])

In [4]:
df_lista = df[['price','neighbourhood_group', 'minimum_nights', 'availability_365','room_type']]

#df_lista['minimum_nights'] = df_lista['minimum_nights'].astype(str)

NameError: name 'df' is not defined

In [ ]:
nonnormal = []
categorical = []

for col in df_lista.columns:
    if df_lista[col].dtype == 'object':
        categorical.append(col)
    else:
        serie = df_lista[col].dropna()
        if len(serie) >= 3:
            if shapiro(serie).pvalue < 0.05:
                nonnormal.append(col)

In [ ]:

TableOne(
    data=df_lista,
    columns=df_lista.columns.tolist(),
    categorical=categorical, 
    nonnormal=nonnormal,
    groupby='room_type',
    pval=True,
    htest_name=True
)

In [ ]:
print(df_lista.columns.tolist())

NameError: name 'df_lista' is not defined

In [ ]:
df_turismo = df[['room_type','price','central_park', 'time_square', 'estatua_libertad', 'broadway']]

In [ ]:

nonnormal = []
categorical = []

for col in df_turismo.columns:
    if df_turismo[col].dtype == 'object':
        categorical.append(col)
    else:
        # Elimina nulos para evitar errores con Shapiro
        datos = df_turismo[col].dropna()
        if len(datos) < 3:
            continue  
        pval = shapiro(datos.sample(min(5000, len(datos))))[1]  # límite por tamaño
        if pval < 0.05:
            nonnormal.append(col)

In [ ]:

TableOne(
    data=df_turismo,
    columns=df_turismo.columns.tolist(),
    categorical=categorical,
    nonnormal=nonnormal,
    groupby='room_type',
    pval=True,
    htest_name=True
)

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(numeric_only=True, method='spearman'), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de correlación')

In [ ]:
correlaciones = df.corr(numeric_only=True, method='spearman')
correlaciones_con_precio = correlaciones['price'].sort_values(key=abs, ascending=False)
print(correlaciones_con_precio)

In [ ]:
correlaciones = df.corr(numeric_only=True, method='pearson')
correlaciones_con_precio = correlaciones['price'].sort_values(key=abs, ascending=False)
print(correlaciones_con_precio)

In [ ]:
sns.boxplot(data=df, x='room_type', y='price')
plt.title('Precio por tipo de habitación')
plt.show()

In [ ]:
df['price'].describe()